In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import dask_geopandas
import dask.dataframe as dd
from shapely import wkt
import warnings
from haversine import haversine, Unit


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

/opt/homebrew/anaconda3/envs/DaskEnv/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
locationWeather = dd.read_csv("data/location-weather-data.csv")
trafficData = pd.read_csv('data/groupedOutput/0.csv')

In [3]:
locationWeather.head()

,zipcode,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,Year,Month,Day,CRASH DATE,BOROUGH,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,COLLISION_ID,Vehicles Involved,Accident Severity,hour
0,10001,4,5,46.1,44.6,48.9,24.0,0.0,5.4,9.9,0,0.0,0,2023,1,1,2023-01-01,MANHATTAN,40.805595,-73.958190,"(40.805595, -73.95819)",0,0,4595061.0,2.0,Extreme Severity,1
1,10001,4,5,46.1,44.6,48.9,24.0,0.0,5.4,9.9,0,0.0,0,2023,1,1,2023-01-01,MANHATTAN,40.758980,-73.962440,"(40.75898, -73.96244)",1,0,4594347.0,2.0,Low Severity,1
2,10001,4,5,46.1,44.6,48.9,24.0,0.0,5.4,9.9,0,0.0,0,2023,1,1,2023-01-01,MANHATTAN,40.795250,-73.973210,"(40.79525, -73.97321)",0,0,4594596.0,1.0,Extreme Severity,7
3,10001,4,5,46.1,44.6,48.9,24.0,0.0,5.4,9.9,0,0.0,0,2023,1,1,2023-01-01,MANHATTAN,40.775936,-73.957115,"(40.775936, -73.957115)",0,0,4594632.0,1.0,Extreme Severity,17
4,10001,4,5,46.1,44.6,48.9,24.0,0.0,5.4,9.9,0,0.0,0,2023,1,1,2023-01-01,MANHATTAN,40.737534,-73.991280,"(40.737534, -73.99128)",1,0,4594774.0,3.0,Moderate Severity,1


In [5]:
def getLat(x):
    return x.split(",")[0]

def getLng(x):
    return x.split(",")[1]

trafficData['Latitude'] = trafficData['LINK_POINTS'].apply(getLat)
trafficData['Longitude'] = trafficData['LINK_POINTS'].apply(getLng)
trafficData['Zipcode'] = trafficData['Zipcode'].astype(int)
trafficDf = dd.from_pandas(trafficData, npartitions = 5)
columns={
    'Zipcode': 'zipcode', 
    'year':	'Year',
    'month': 'Month',
    'date' : 'Day'
    }

trafficDf = trafficDf.rename(columns=columns)
trafficData.head()

,year,month,date,hour,LINK_POINTS,Zipcode,BOROUGH,SPEED,Latitude,Longitude
0,2023,12,31,11,"40.80151,-73.93066",10035,Manhattan,26.417619,40.80151,-73.93066
1,2023,12,31,11,"40.8014,-73.93111",10035,Manhattan,26.417619,40.8014,-73.93111
2,2023,12,31,11,"40.8012304,-73.93129",10035,Manhattan,26.417619,40.8012304,-73.93129
3,2023,12,31,11,"40.80096,-73.93141",10035,Manhattan,26.417619,40.80096,-73.93141
4,2023,12,31,11,"40.8007405,-73.93133",10035,Manhattan,26.417619,40.8007405,-73.93133


In [7]:
locationWeather = locationWeather.repartition(npartitions=20)

In [8]:
mergedDf = locationWeather.merge(trafficDf, on = ['zipcode','Year','Month','Day','hour'], how = "left")
mergedDf.head()

,zipcode,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,Year,Month,Day,CRASH DATE,BOROUGH_x,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,COLLISION_ID,Vehicles Involved,Accident Severity,hour,LINK_POINTS,BOROUGH_y,SPEED,Latitude,Longitude
0,11363,2,4,48.6,41.0,57.9,36.3,0.31,14.3,27.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.789680,-73.826096,"(40.78968, -73.826096)",1,0,4685558.0,1.0,Low Severity,13,<NA>,<NA>,NaN,<NA>,<NA>
1,11363,2,4,48.6,41.0,57.9,36.3,0.31,14.3,27.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.723534,-73.754250,"(40.723534, -73.75425)",0,0,4685090.0,3.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>
2,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.726696,-73.819060,"(40.726696, -73.81906)",0,0,4685649.0,2.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>
3,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.725285,-73.793330,"(40.725285, -73.79333)",0,0,4685012.0,2.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>
4,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.789680,-73.826096,"(40.78968, -73.826096)",1,0,4685558.0,1.0,Low Severity,13,<NA>,<NA>,NaN,<NA>,<NA>


In [11]:
def getDistance(row):
    if pd.isna(row['LATITUDE']) or pd.isna(row['LONGITUDE']) or pd.isna(row['Latitude']) or pd.isna(row['Longitude']):
        return 0.0
    return haversine((row['LATITUDE'],row['LONGITUDE']),(float(row['Latitude']),float(row['Longitude'])), unit = 'm')

mergedDf['distances'] = mergedDf.apply(lambda x : getDistance(x), axis = 1, meta = float)
mergedDf.to_csv("withDistances/*.csv", index = False)

['/Users/neelgandhi/Big Dat/Final Project/withDistances/00.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/01.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/02.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/03.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/04.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/05.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/06.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/07.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/08.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/09.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/10.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/11.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/12.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/13.csv',
 '/Users/neelgandhi/Big Dat/Final Project/withDistances/14.csv',
 '/Users/neelgandhi/Big D

In [26]:
dtypes = mergedDf.dtypes

In [29]:
dtypes.to_dict()

{'zipcode': dtype('int64'),
 'school_count': dtype('int64'),
 'park_count': dtype('int64'),
 'MeanTemp': dtype('float64'),
 'MinTemp': dtype('float64'),
 'MaxTemp': dtype('float64'),
 'DewPoint': dtype('float64'),
 'Percipitation': dtype('float64'),
 'WindSpeed': dtype('float64'),
 'MaxSustainedWind': dtype('float64'),
 'Rain': dtype('int64'),
 'SnowDepth': dtype('float64'),
 'SnowIce': dtype('int64'),
 'Year': dtype('int64'),
 'Month': dtype('int64'),
 'Day': dtype('int64'),
 'CRASH DATE': string[pyarrow],
 'BOROUGH_x': string[pyarrow],
 'LATITUDE': dtype('float64'),
 'LONGITUDE': dtype('float64'),
 'LOCATION': string[pyarrow],
 'NUMBER OF PERSONS INJURED': dtype('int64'),
 'NUMBER OF PERSONS KILLED': dtype('int64'),
 'COLLISION_ID': dtype('float64'),
 'Vehicles Involved': dtype('float64'),
 'Accident Severity': string[pyarrow],
 'hour': dtype('int64'),
 'LINK_POINTS': string[pyarrow],
 'BOROUGH_y': string[pyarrow],
 'SPEED': dtype('float64'),
 'Latitude': string[pyarrow],
 'Longitude

In [30]:
mdf = dd.read_csv("withDistances/*.csv", dtype=dtypes.to_dict())

In [32]:
mdf[mdf['distances']<=300].to_csv("newMergedData/*.csv",index = False)

['/Users/neelgandhi/Big Dat/Final Project/newMergedData/00.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/01.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/02.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/03.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/04.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/05.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/06.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/07.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/08.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/09.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/10.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/11.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/12.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/13.csv',
 '/Users/neelgandhi/Big Dat/Final Project/newMergedData/14.csv',
 '/Users/neelgandhi/Big D

In [37]:
md = dd.read_csv('newMergedData/*.csv', dtype=dtypes.to_dict())


In [38]:
md.head()

,zipcode,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,Year,Month,Day,CRASH DATE,BOROUGH_x,LATITUDE,LONGITUDE,LOCATION,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,COLLISION_ID,Vehicles Involved,Accident Severity,hour,LINK_POINTS,BOROUGH_y,SPEED,Latitude,Longitude,distances
0,11363,2,4,48.6,41.0,57.9,36.3,0.31,14.3,27.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.789680,-73.826096,"(40.78968, -73.826096)",1,0,4685558.0,1.0,Low Severity,13,<NA>,<NA>,NaN,<NA>,<NA>,0.0
1,11363,2,4,48.6,41.0,57.9,36.3,0.31,14.3,27.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.723534,-73.754250,"(40.723534, -73.75425)",0,0,4685090.0,3.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>,0.0
2,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.726696,-73.819060,"(40.726696, -73.81906)",0,0,4685649.0,2.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>,0.0
3,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.725285,-73.793330,"(40.725285, -73.79333)",0,0,4685012.0,2.0,Extreme Severity,13,<NA>,<NA>,NaN,<NA>,<NA>,0.0
4,11363,2,4,50.1,41.0,60.1,34.3,0.69,11.9,19.0,1,0.0,0,2023,12,6,2023-12-06,QUEENS,40.789680,-73.826096,"(40.78968, -73.826096)",1,0,4685558.0,1.0,Low Severity,13,<NA>,<NA>,NaN,<NA>,<NA>,0.0


In [40]:
# ['school_count','park_count','MeanTemp','MinTemp','MaxTemp','DewPoint','Percipitation','WindSpeed','MaxSustainedWind','Rain','SnowDepth','SnowIce']
newData = md.groupby(['COLLISION_ID','Year','Month','Day','hour','LATITUDE','LONGITUDE','BOROUGH_x']).agg({
    'zipcode' : 'first',
    'SPEED' : 'mean',
    'school_count' : 'mean',
    'park_count' : 'mean',
    'MeanTemp' : 'mean',
    'MinTemp' : 'mean',
    'MaxTemp' : 'mean',
    'DewPoint' : 'mean',
    'Percipitation' : 'mean',
    'WindSpeed' : 'mean',
    'MaxSustainedWind' : 'mean',
    'Rain' : 'mean',
    'SnowDepth' : 'mean',
    'SnowIce' : 'mean',
    'NUMBER OF PERSONS INJURED'	: 'mean',
    'NUMBER OF PERSONS KILLED' : 'mean', 
    'Accident Severity': 'first'
})
newData = newData.reset_index()
newData.head()

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity


In [51]:
newData.head(30)

,COLLISION_ID,Year,Month,Day,hour,LATITUDE,LONGITUDE,BOROUGH_x,zipcode,SPEED,school_count,park_count,MeanTemp,MinTemp,MaxTemp,DewPoint,Percipitation,WindSpeed,MaxSustainedWind,Rain,SnowDepth,SnowIce,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,Accident Severity
0,4685558.0,2023,12,6,13,40.789680,-73.826096,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity
1,4685090.0,2023,12,6,13,40.723534,-73.754250,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
2,4685649.0,2023,12,6,13,40.726696,-73.819060,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
3,4685012.0,2023,12,6,13,40.725285,-73.793330,QUEENS,11363,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
4,4685232.0,2023,12,6,20,40.716140,-73.833570,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
5,4685560.0,2023,12,6,20,40.781680,-73.847770,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
6,4685289.0,2023,12,6,20,40.687294,-73.807304,QUEENS,11364,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,1.0,0.0,Low Severity
7,4686217.0,2023,12,6,8,40.759700,-73.800310,QUEENS,11365,NaN,5.352941,7.647059,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
8,4686477.0,2023,12,6,23,40.768726,-73.900240,QUEENS,11366,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity
9,4685142.0,2023,12,6,17,40.711433,-73.787680,QUEENS,11368,NaN,5.929825,7.754386,49.35,41.0,59.0,35.3,0.5,13.1,23.0,1.0,0.0,0.0,0.0,0.0,Extreme Severity


In [42]:
newData.to_csv('newFinal/*.csv', index = False)

['/Users/neelgandhi/Big Dat/Final Project/newFinal/0.csv']

In [43]:
final = pd.read_csv("newFinal/0.csv")